# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [2]:
client_key = "bIJz13N9uTSt9lJLhsNNpkB9f"
client_secret =  "xe50C8MS7Z45lvyQFOUI1DoZzddyiiKj1lFpeu7Uo5zrfk6iEp"

In [3]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [4]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [5]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [6]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAJ4megEAAAAA94JEOvq9csaKixevvuE6SfWlXTo%3DKJh1dOpFT75BC8iziEhGWnIm7pEy5teQcIdHIYbH5z0MD1AyRd'}

In [7]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [8]:
max_tweets=100
tw_limit=100  # we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [9]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [10]:
climate_tweets = response.json()


In [11]:
len(climate_tweets['data'])

100

In [12]:
tweets +=climate_tweets['data']


In [13]:
ids = [tw['id'] for tw in climate_tweets['data']]

In [14]:
min_id = min(ids)

In [15]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [16]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'until_id': min_id,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
                 }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [17]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 100


# Let's print some tweets

In [18]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [19]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '532256657',
  'conversation_id': '1549587618551062528',
  'created_at': '2022-07-20T02:50:52.000Z',
  'entities': {'mentions': [{'end': 18,
                             'id': '18089606',
                             'start': 3,
                             'username': 'KurtSchlichter'}]},
  'id': '1549587618551062528',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 328},
  'referenced_tweets': [{'id': '1549549544681050112', 'type': 'retweeted'}],
  'reply_settings': 'everyone',
  'source': 'Twitter for iPad',
  'text': 'RT @KurtSchlichter: 👏Climate👏change👏is👏a👏hoax👏'},
 {'author_id': '976078973418446848',
  'conversation_id': '1549587617443975168',
  'created_at': '2022-07-20T02:50:52.000Z',
  'entities': {'mentions': [{'end': 11,
                             'id': '34317032',
                             'start': 3,
                             'us

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [20]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

RT @KurtSchlichter: 👏Climate👏change👏is👏a👏hoax👏

RT @KHayhoe: How does climate change affect heatwaves? A thread.
1 - As the planet warms, extreme heat now begins earlier in the year and s…

RT @hrdtimez: and every day people are the ones that are being told to “tackle climate change”

They are fast tracking their agendas.  Trying to hurry up with plandemics, climate change etc to take guns, so you are defenseless on what they're about to do to you and your families #Agenda2030 https://t.co/KLc8IrfPuA



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [21]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['author_id', 'entities', 'reply_settings', 'lang', 'id', 'created_at', 'public_metrics', 'text', 'source', 'conversation_id', 'referenced_tweets'])
11


In [22]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [23]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here
        
        
        yield modified_tweet       
            
            

In [24]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [25]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'entities': {'mentions': [{'start': 3,
     'end': 18,
     'username': 'KurtSchlichter',
     'id': '18089606'}]},
  'reply_settings': 'everyone',
  'lang': 'en',
  'id': '1549587618551062528',
  'created_at': '2022-07-20T02:50:52.000Z',
  'public_metrics': {'retweet_count': 328,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'text': 'RT @KurtSchlichter: 👏Climate👏change👏is👏a👏hoax👏',
  'source': 'Twitter for iPad',
  'conversation_id': '1549587618551062528',
  'referenced_tweets': [{'type': 'retweeted', 'id': '1549549544681050112'}],
  'user_id': '532256657',
  'referenced_tweets_ids': '1549549544681050112',
  'retweet_count': 328},
 {'entities': {'mentions': [{'start': 3,
     'end': 11,
     'username': 'KHayhoe',
     'id': '34317032'}]},
  'reply_settings': 'everyone',
  'lang': 'en',
  'id': '1549587617443975168',
  'created_at': '2022-07-20T02:50:52.000Z',
  'public_metrics': {'retweet_count': 505,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},

In [26]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [27]:
climate_feed_df

,entities,reply_settings,lang,id,created_at,public_metrics,text,source,conversation_id,referenced_tweets,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id
0,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",everyone,en,1549587618551062528,2022-07-20T02:50:52.000Z,"{'retweet_count': 328, 'reply_count': 0, 'like...",RT @KurtSchlichter: 👏Climate👏change👏is👏a👏hoax👏,Twitter for iPad,1549587618551062528,"[{'type': 'retweeted', 'id': '1549549544681050...",532256657,1549549544681050112,328,NaN
1,"{'mentions': [{'start': 3, 'end': 11, 'usernam...",everyone,en,1549587617443975168,2022-07-20T02:50:52.000Z,"{'retweet_count': 505, 'reply_count': 0, 'like...",RT @KHayhoe: How does climate change affect he...,Twitter for iPhone,1549587617443975168,"[{'type': 'retweeted', 'id': '1549459174844731...",976078973418446848,1549459174844731392,505,NaN
2,"{'mentions': [{'start': 3, 'end': 12, 'usernam...",everyone,en,1549587616546168832,2022-07-20T02:50:52.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",RT @hrdtimez: and every day people are the one...,Twitter for iPhone,1549587616546168832,"[{'type': 'retweeted', 'id': '1549557847876595...",1516374858,1549557847876595713,1,NaN
3,"{'hashtags': [{'start': 184, 'end': 195, 'tag'...",everyone,en,1549587612582780928,2022-07-20T02:50:51.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",They are fast tracking their agendas. Trying ...,Twitter for Android,1549587612582780928,"[{'type': 'quoted', 'id': '1549123326244798466'}]",1285057671058591746,1549123326244798466,0,NaN
4,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",everyone,en,1549587609302831104,2022-07-20T02:50:50.000Z,"{'retweet_count': 1981, 'reply_count': 0, 'lik...",RT @KurtSchlichter: Climate change is a hoax d...,Twitter for iPhone,1549587609302831104,"[{'type': 'retweeted', 'id': '1549389251149709...",565673259,1549389251149709312,1981,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,"{'mentions': [{'start': 3, 'end': 11, 'usernam...",everyone,en,1549587373511540737,2022-07-20T02:49:54.000Z,"{'retweet_count': 46, 'reply_count': 0, 'like_...",RT @joshgad: To look around and see what’s hap...,Twitter for iPhone,1549587373511540737,"[{'type': 'retweeted', 'id': '1549474946035765...",930534103,1549474946035765248,46,NaN
86,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",everyone,en,1549587372672962560,2022-07-20T02:49:54.000Z,"{'retweet_count': 8, 'reply_count': 0, 'like_c...",RT @CatInthehat002: Some of the worlds “elites...,Twitter Web App,1549587372672962560,"[{'type': 'retweeted', 'id': '1549297077255294...",1540911226540400640,1549297077255294977,8,NaN
87,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",everyone,en,1549587371145961472,2022-07-20T02:49:53.000Z,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",RT @Climatepledgeco: Want to support Ukraine a...,Twitter Web App,1549587371145961472,"[{'type': 'retweeted', 'id': '1549572165317558...",291212837,1549572165317558273,3,NaN
88,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",everyone,en,1549587370084810753,2022-07-20T02:49:53.000Z,"{'retweet_count': 239, 'reply_count': 0, 'like...","RT @MattH_4America: ""Climate Change"" is a hoax...",Twitter for Android,1549587370084810753,"[{'type': 'retweeted', 'id': '1549544142048468...",866348764432392192,1549544142048468998,239,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [28]:
# write code here datatype listing

climate_feed_df.dtypes

entities                 object
reply_settings           object
lang                     object
id                       object
created_at               object
public_metrics           object
text                     object
source                   object
conversation_id          object
referenced_tweets        object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
in_reply_to_user_id      object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

All IDs should be either strings or integers. Text should be a string. 


# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [29]:
def tweet_user_generator(tweet_user_list):
    for user in tweet_user_list:
        try:
            yield user
        except KeyError:
            continue


In [30]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [31]:
user_df

,0
0,data
1,includes
2,meta


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [32]:
user_df.dtypes

0    object
dtype: object

# write columns with wrong datatype and mention correct data type too.

This is a bit tricky since question 4 doesn't really do anything. My guess that column 0 should be a string. 

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [33]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [34]:
retweeted_status_df.head()

,id,retweet_count
0,1549587618551062528,328
1,1549587617443975168,505
2,1549587616546168832,1
3,1549587612582780928,0
4,1549587609302831104,1981


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [37]:
# Write code here

not_retweeted = (retweeted_status_df['retweet_count'] == 0).sum()
print(not_retweeted)

14
